In [ ]:
import sopa
import spatialdata

#sdata = spatialdata.read_zarr("/Volumes/DATA_2025/Projects/RAO/dbs/Xenium_and_AdjustedPhenoCycler.V6.zarr/").query.bounding_box(
#    axes=("x", "y"),
#    min_coordinate=[13000, 12000],
#    max_coordinate=[14500, 13000],
#    target_coordinate_system="global"
#)
#sdata


### In order to plot beforehand prepared sdata ###
sdata = spatialdata.read_zarr("/Volumes/DATA_2025/Projects/RAO/dbs/Xenium_and_AdjustedPhenoCycler.V6.SubSectioned.SegmentationAttempts.zarr/")


In [ ]:
#sdata = sdata_main.query.bounding_box(
#    axes=("x", "y"),
#    min_coordinate=[13000, 12000],
#    max_coordinate=[14500, 13000],
#    target_coordinate_system="global"
#)
#sdata
#sdata_main = None

In [ ]:
#import spatialdata_plot      # register the .pl accessor

# Render just the "Xenium_Image" element, then display it
#sdata.pl.render_images(element="Xenium_Image") \
#     .pl.show()

#sdata.pl.render_images(element="HE") \
#     .pl.show()

In [ ]:
# Render just the "Xenium_Image" element, then display it
#sdata.pl.render_images(element="Xenium_Image") \
#     .pl.show()

#sdata.pl.render_shapes(element="cell_boundaries") \
#     .pl.show()

# Segmentation High-Througput is started

In [ ]:
# CELLPOSE SEGMENTATION
import spatialdata_plot  

diameters = [25, 30, 35 , 40, 45]
flow_thresholds = [1, 2 , 3]
cellprob_thresholds = [-10 ,-8, -6, -4, -1, 0]
gaussians = [0, 1, 2]
#clip_limit=0.0
base_save_dir = "/Volumes/DATA_2025/Projects/RAO/dbs/comparison_attempt"

# Make sure the save directory exists
import os
os.makedirs(base_save_dir, exist_ok=True)

for gaus in gaussians:
    for diameter in diameters:
        for flow in flow_thresholds:
            for cp in cellprob_thresholds:
                # 1) build a descriptive name for this attempt
                attempt_info = f"cellpose_DAPI_CytoMarkers_d{diameter}_ft{int(flow)}_cp{int(abs(cp))}_gaus{int(abs(gaus))}"
                print(f"\n--- Running attempt: {attempt_info} ---")
                if attempt_info not in sdata.shapes.keys():
                    sopa.segmentation.tissue(sdata, image_key="Xenium_Image")
                    #sopa.make_image_patches(sdata)
                    #sopa.utils.get_channel_names(sdata)
                    sopa.make_image_patches(sdata, image_key="Xenium_Image")

        
        
                    sopa.settings.parallelization_backend = "dask"
                    sopa.settings.dask_client_kwargs["n_workers"] = 4
        
                    #2) run segmentation with that key_added
                    sopa.segmentation.cellpose(
                        sdata,
                        channels=["DAPI","ATP1A1/CD45/E-Cadherin"],
                        diameter=diameter,
                        image_key="Xenium_Image",
                        model_type="cyto2",
                        flow_threshold=flow,
                        cellprob_threshold=cp,
                        clip_limit=0.0,
                        gaussian_sigma=gaus,
                        key_added=attempt_info
                    )
        
                    # 3) render & save the shapes to SVG
                    sdata \
                    .pl.render_shapes(attempt_info, color="red", fill_alpha=0.5) \
                    .pl.render_shapes("cell_boundaries", color="blue", fill_alpha=0.5) \
                    .pl.show(
                        coordinate_systems="global",
                        save=os.path.join(base_save_dir, f"{attempt_info}.png"),
                        dpi=800
                    )
                else:
                    print(f"Attempt {attempt_info} already exists, skipping...")




In [ ]:
#BAYSOR SEGMENTATION
scales = [2, 3 ,4, 5]
minAreas = [0, 5, 10]

base_save_dir = "/Volumes/DATA_2025/Projects/RAO/dbs/comparison_attempt"

# Make sure the save directory exists
import os
os.makedirs(base_save_dir, exist_ok=True)

for scle in scales:
    for minArea in minAreas:
        
                # 1) build a descriptive name for this attempt
                attempt_info = f"baysor_XeniumStock_scale{scle}_minArea{int(minArea)}"
                print(f"\n--- Running attempt: {attempt_info} ---")
    
                # sopa.make_transcript_patches(sdata, patch_width=500, prior_shapes_key="cell_boundaries")

                # sopa.settings.parallelization_backend = "dask"
                # sopa.settings.dask_client_kwargs["n_workers"] = 4
    
                # # 2) run segmentation with that key_added
                # sopa.segmentation.baysor(
                #     sdata,
                #     scale=scle,
                #     min_area=minArea,
                #     key_added=attempt_info
                # )
    
                # 3) render & save the shapes to SVG
                sdata \
                   .pl.render_shapes(attempt_info, color="red", fill_alpha=0.5) \
                   .pl.render_shapes("cell_boundaries", color="blue", fill_alpha=0.5) \
                   .pl.show(
                       coordinate_systems="global",
                       save=os.path.join(base_save_dir, f"{attempt_info}.png"),
                       dpi=800
                   )


In [ ]:
#PROSEG SEGMENTATION


base_save_dir = "/Volumes/DATA_2025/Projects/RAO/dbs/comparison_attempt"

# Make sure the save directory exists
import os
os.makedirs(base_save_dir, exist_ok=True)


# sopa.make_transcript_patches(sdata, patch_width=None, prior_shapes_key="cell_boundaries")
# sopa.segmentation.proseg(sdata)
    
# 3) render & save the shapes to SVG
sdata \
   .pl.render_shapes("proseg_boundaries", color="red", fill_alpha=0.5) \
   .pl.render_shapes("cell_boundaries", color="blue", fill_alpha=0.5) \
   .pl.show(
       coordinate_systems="global",
       save=os.path.join(base_save_dir, f"{attempt_info}.png"),
       dpi=800
   )




In [ ]:
#STARDIST SEGMENTATION (H&E)

probTs = [0.01 ,0.2, 0.4 , 0.6]
minAreas = [0, 5]
nmsTs = [0.2, 0.6, 0.9, 2 , 4, 6]
base_save_dir = "/Users/ugursahin/RAO/segmentationOptimizationAttempts"

# Make sure the save directory exists
import os
os.makedirs(base_save_dir, exist_ok=True)

for probT in probTs:
    for nmsT in nmsTs:
        for minArea in minAreas:
                # 1) build a descriptive name for this attempt
                attempt_info = f"stardist_versatileHE_XeniumStock_probT{probT}_nmsT{nmsT}_minArea{int(minArea)}"
                print(f"\n--- Running attempt: {attempt_info} ---")
                if attempt_info not in sdata.shapes.keys():                

                    sopa.settings.parallelization_backend = "dask"
                    sopa.settings.dask_client_kwargs["n_workers"] = 4
        
                    #2) run segmentation with that key_added
                    sopa.make_image_patches(sdata, image_key="HE", patch_width=1500, patch_overlap=50)
                    sopa.make_transcript_patches(sdata, patch_width=None, prior_shapes_key="cell_boundaries")

                    sopa.segmentation.stardist(sdata,image_key="HE", 
                                               model_type='2D_versatile_he',
                                               min_area=minArea,
                                               prob_thresh=probT,
                                               nms_thresh=nmsT,
                                               key_added=attempt_info)
        
                    # 3) render & save the shapes to SVG
                    sdata \
                    .pl.render_shapes(attempt_info, color="red", fill_alpha=0.5) \
                    .pl.render_shapes("cell_boundaries", color="blue", fill_alpha=0.5) \
                    .pl.show(
                        coordinate_systems="global",
                        save=os.path.join(base_save_dir, f"{attempt_info}.png"),
                        dpi=800
                    )
                else:
                    print(f"Attempt {attempt_info} already exists, skipping...")


In [ ]:
#STARDIST SEGMENTATION (FLUORESCENCE)

probTs = [0.01 ,0.2, 0.4]
minAreas = [0, 5]
nmsTs = [0.2, 0.6, 0.9, 2 , 4, 6]
base_save_dir = "/Users/ugursahin/RAO/segmentationOptimizationAttempts"

# Make sure the save directory exists
import os
os.makedirs(base_save_dir, exist_ok=True)

for probT in probTs:
    for nmsT in nmsTs:
        for minArea in minAreas:
            
                    # 1) build a descriptive name for this attempt
                    attempt_info = f"stardist_versatileFluo_XeniumStock_probT{probT}_nmsT{nmsT}_minArea{int(minArea)}"
                    print(f"\n--- Running attempt: {attempt_info} ---")
                    if attempt_info not in sdata.shapes.keys():        

                        sopa.settings.parallelization_backend = "dask"
                        sopa.settings.dask_client_kwargs["n_workers"] = 4
            
                        #2) run segmentation with that key_added
                        sopa.make_image_patches(sdata, image_key="Xenium_Image", patch_width=1500, patch_overlap=50)
                        sopa.make_transcript_patches(sdata, patch_width=None, prior_shapes_key="cell_boundaries")

                        sopa.segmentation.stardist(sdata,image_key="Xenium_Image", 
                                                model_type='2D_versatile_fluo',
                                                channels=["DAPI"],
                                                min_area=minArea,
                                                prob_thresh=probT,
                                                nms_thresh=nmsT,
                                                key_added=attempt_info)
            
                        # 3) render & save the shapes to SVG
                        sdata \
                        .pl.render_shapes(attempt_info, color="red", fill_alpha=0.5) \
                        .pl.render_shapes("cell_boundaries", color="blue", fill_alpha=0.5) \
                        .pl.show(
                            coordinate_systems="global",
                            save=os.path.join(base_save_dir, f"{attempt_info}.png"),
                            dpi=800
                        )
                    else:
                        print(f"Attempt {attempt_info} already exists, skipping...")


In [ ]:
sdata.write("/Volumes/DATA_2025/Projects/RAO/dbs/Xenium_and_AdjustedPhenoCycler.V6.SubSectioned.SegmentationAttempts.V2.zarr/")